In [1]:
import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import tqdm
from collections import deque

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from torch.distributions import Categorical

import matplotlib.pyplot as plt
import numpy as np

In [2]:
NUM_EPISODES = 10000
MAX_STEPS = 200
RECENT_EPS = 20
SOLVED_SCORE = 0

GAMMA = 0.99
HIDDEN_UNITS = 256
LR_ACTOR = 0.0005
LR_CRITIC = 0.0005

#device to run model on 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cpu


In [3]:
class Actor(nn.Module):    
    def __init__(self, observation_space, hidden_units, action_space):
        super(Actor, self).__init__()
        self.input_layer = nn.Linear(observation_space, hidden_units)
        self.output_layer = nn.Linear(hidden_units, action_space)
    
    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        actions = self.output_layer(x)
        action_probs = F.softmax(actions, dim=1)
        return action_probs

In [4]:
class Critic(nn.Module):
    def __init__(self, observation_space, hidden_units):
        super(Critic, self).__init__()
        self.input_layer = nn.Linear(observation_space, hidden_units)
        self.output_layer = nn.Linear(hidden_units, 1)
        
    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        state_value = self.output_layer(x)
        return state_value

In [5]:
env = gym.make("Pong-ramDeterministic-v4", obs_type="ram")

observation_space = env.observation_space.shape[0]
action_space = env.action_space.n
hidden_units = HIDDEN_UNITS

actor = Actor(observation_space, hidden_units, action_space).to(DEVICE)
critic = Critic(observation_space, hidden_units).to(DEVICE)

actor_optimizer = torch.optim.Adam(actor.parameters(), lr=LR_ACTOR)
critic_optimizer = torch.optim.Adam(critic.parameters(), lr=LR_CRITIC)

In [6]:
def select_action(actor, state):
    ''' Selects an action given current state using the actor network.'''
    
    #convert state to float tensor, add 1 dimension, allocate tensor on device
    state = torch.from_numpy(state).float().unsqueeze(0).to(DEVICE)
    
    #use network to predict action probabilities
    action_probs = actor(state)
    state = state.detach()
    
    #sample an action using the probability distribution
    m = Categorical(action_probs)
    action = m.sample()
    
    #return action
    return action.item(), m.log_prob(action)

In [7]:
#track scores
scores = []

#track recent scores
recent_scores = deque(maxlen = RECENT_EPS)

#run episodes
for episode in tqdm.trange(NUM_EPISODES):
    
    #init variables
    state, _ = env.reset()
    done = False
    score = 0
    I = 1
    
    #run episode, update online
    for step in range(MAX_STEPS):
        
        #get action and log probability
        action, lp = select_action(actor, state)
        
        #step with action
        new_state, reward, done, _, _ = env.step(action)
        
        #update episode score
        score += reward
        
        #get state value of current state
        state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(DEVICE)
        state_val = critic(state_tensor)
        
        #get state value of next state
        new_state_tensor = torch.from_numpy(new_state).float().unsqueeze(0).to(DEVICE)        
        new_state_val = critic(new_state_tensor)
        
        #if terminal state, next state val is 0
        if done:
            new_state_val = torch.tensor([0]).float().unsqueeze(0).to(DEVICE)
        
        #calculate value function loss with MSE
        val_loss = F.smooth_l1_loss(reward + GAMMA * new_state_val, state_val)
        val_loss *= I
        
        #calculate policy loss
        advantage = reward + GAMMA * new_state_val.item() - state_val.item()
        actor_loss = -lp * advantage
        actor_loss *= I
        
        #Backpropagate actor
        actor_optimizer.zero_grad()
        actor_loss.backward(retain_graph=True)
        actor_optimizer.step()
        
        #Backpropagate critic
        critic_optimizer.zero_grad()
        val_loss.backward()
        critic_optimizer.step()
        
        if done:
            break
            
        #move into new state, discount I
        state = new_state
        I *= GAMMA
    
    #append episode score 
    scores.append(score)
    recent_scores.append(score)

    #print episode stats
    if episode % RECENT_EPS == 0:
        print('Episode {}\tAverage Score: {:.2f}'.format(episode, np.array(recent_scores).mean()))
    
    #early stopping if we meet solved score goal
    if np.array(recent_scores).mean() >= SOLVED_SCORE:
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 1/10000 [00:01<3:14:11,  1.17s/it]

Episode 0	Average Score: -4.00


  0%|          | 21/10000 [00:37<5:14:55,  1.89s/it]

Episode 20	Average Score: -4.00


...

Episode 640	Average Score: -4.00


  7%|▋         | 661/10000 [17:54<3:51:32,  1.49s/it]

Episode 660	Average Score: -4.00


  7%|▋         | 679/10000 [18:32<4:14:33,  1.64s/it]


KeyboardInterrupt: 